In [ ]:
!pip install python-docx
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract
!pip install opencv-python
!pip install date-extractor
!pip install PyMuPDF
!pip install ipywidgets
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

# Tesseract

In [1]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import pandas as pd

# Set the Tesseract command path after installation
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

## For single pdf

In [4]:

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
pdf_path = r'C:\Users\Aveno\Documents\Extract-Resumes-using-Tesseract\cv\pdf\1042050.pdf'
images = convert_from_path(pdf_path)
for i, image in enumerate(images):
    text = pytesseract.image_to_string(image)


In [5]:
text

'DARKO\n\nKESEGI\n\nMSc of Economics\n\nBIRTH DATE:\n15.02.1996.\n\nCITY:\nSubotica, Serbia\n\nEMAIL:\ndarkokesegi@gmail.com\n\nNUMBER:\n+381 60 0552 608\n\n+381 63 596 383 G=>\n\nLinkedin:\nwww.linkedin.com/in/darko-kesegi\n\ne EDUCATION\n\nFaculty of Economics in Subotica, University\nof Novi\n\nSad\n\n“Master studies\n\nDirection: Finance, Banking and Insurance\n-enrolled / finished: 2019 /2020\n“Undergraduate\n\n-enrolled / finished: 2015/2019\n\nHigh School of Economics "Bosa Miliéevi¢",\nSubotica\n\n-direction: economic technician\n-enrolled / finished: 2011/2015\n\n« LANGUAGES\n- Serbian - Native\n- English - Advanced\n\n- German - Intermediate\n\n¢ CERTIFICATION\n-ECDL certificate\n-Excel: Advanced formulas and Functions\n(Linkedin learning)\n\n¢ PERSONAL PROFILE\n| think for myself that | am very organized, responsible, persistent and hardworking person.\n| am able to handle multiple tasks in daily basis. | shown myself to be a person capable of\ncooperating with people of var

## For all pdfs in the path

In [6]:
pdf_dir = r'C:\Users\Aveno\Documents\Extract-Resumes-using-Tesseract\cv\pdf'
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
df = pd.DataFrame(columns=['filename', 'text'])


for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    images = convert_from_path(pdf_path)
    text_content = ''
    for image in images:
        text = pytesseract.image_to_string(image)
        text_content += text + ' '  # Add a space as a separator between pages

    new_row = pd.DataFrame({'filename': [pdf_file], 'text': [text_content]})
    df = pd.concat([df, new_row], ignore_index=True)

df


,filename,text
0,1034151.pdf,"Hovhannes Mkrtchyan\n\nGeyerstrasse 52, 80469,..."
1,1037402.pdf,"Birthday: August 26th, 1993\n\nHome base: Inns..."
2,1039995.pdf,Nicholas Jedamzik\n\nn.jedamzik@jedamzik-jedam...
3,1042050.pdf,DARKO\n\nKESEGI\n\nMSc of Economics\n\nBIRTH D...
4,1042140.pdf,Einsatzschwerpunkte\n\nConsulting\n\nMentoring...
5,1042449.pdf,SAMI RAUSCH\n\nManagement Consultant\n\nPERSON...
6,1042894.pdf,Adresse:\n\nGiesestraBe 60\n12621 Berlin\n\nMi...
7,1043487.pdf,LEBENSLAUF Eunsu AHN\n\nPERSONLICHE DATEN\n\nA...
8,1044442.pdf,Top Itservices AG Borhane Cheikh Ahmed\n\nBrii...
9,example.pdf,Lebenslauf Lisa Maria Zettier\n\nPersonliche D...


In [2]:
# df.to_csv('df_tesseract.csv')
df = pd.read_csv('df_tesseract.csv')

In [5]:
import spacy
import pandas as pd
import re


nlp = spacy.load('en_core_web_trf')
phone_regex = re.compile(r'\b(?:\+?\d{1,3})?\s*(?:\(\d{2,3}\)\s*)?\d{3}[-\s]?\d{2,3}[-\s]?\d{2,4}\b')
email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')


def extract_phone_email(text):
    phones = phone_regex.findall(text)
    emails = email_regex.findall(text)
    return phones, emails

def extract_emails_with_spacy(text):
    doc = nlp(text)
    return [token.text for token in doc if token.like_email]

def extract_names(text):
    doc = nlp(text)
    names = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    return names

df['phones'], df['emails'] = zip(*df['text'].apply(extract_phone_email))
df['emails_spacy'] = df['text'].apply(extract_emails_with_spacy)
df['names'] = df['text'].apply(extract_names)

df


,Unnamed: 0,filename,text,phones,emails,emails_spacy,names
0,0,1034151.pdf,"Hovhannes Mkrtchyan\n\nGeyerstrasse 52, 80469,...",[176 77512727],[mkrtchyan.hov.hm@gmail.com],[mkrtchyan.hov.hm@gmail.com],[Hovhannes Mkrtchyan]
1,1,1037402.pdf,"Birthday: August 26th, 1993\n\nHome base: Inns...",[43 676 83555],"[juergen.hahn@gisolutions.at, markus.salmhofer...","[juergen.hahn@gisolutions.at, markus.salmhofer...","[Judith Kuneth, Judith Kuneth, Jurgen Hahn, Ma..."
2,2,1039995.pdf,Nicholas Jedamzik\n\nn.jedamzik@jedamzik-jedam...,[58609041],[n.jedamzik@jedamzik-jedamzik.de],[n.jedamzik@jedamzik-jedamzik.de],"[Nicholas Jedamzik, Marz 2022\nBTC]"
3,3,1042050.pdf,DARKO\n\nKESEGI\n\nMSc of Economics\n\nBIRTH D...,"[381 60 0552, 381 63 596]",[darkokesegi@gmail.com],[darkokesegi@gmail.com],[]
4,4,1042140.pdf,Einsatzschwerpunkte\n\nConsulting\n\nMentoring...,[2021-2022],"[info@digitalfrontiers.de, info@digitalfrontie...","[info@digitalfrontiers.de, info@digitalfrontie...","[Herrn Jerat, Herr Jerat, Herr Jerat, Im Zuge ..."
5,5,1042449.pdf,SAMI RAUSCH\n\nManagement Consultant\n\nPERSON...,[49 170 174 58],[sami.rausch@consulting-rausch.com],[sami.rausch@consulting-rausch.com],[Carve-Out]
6,6,1042894.pdf,Adresse:\n\nGiesestraBe 60\n12621 Berlin\n\nMi...,"[030-50 56, 030-50 56, 030-50 56, 030-50 56, 0...","[NeumannMirko@gmx.de, NeumannMirko@gmx.de, Neu...","[NeumannMirko@gmx.de, NeumannMirko@gmx.de, Neu...","[Mirko Neumann, Mirko Neumann, Mirko Neumann, ..."
7,7,1043487.pdf,LEBENSLAUF Eunsu AHN\n\nPERSONLICHE DATEN\n\nA...,[157 58219873],[eunsuahn.pro@gmail.com],[eunsuahn.pro@gmail.com],[]
8,8,1044442.pdf,Top Itservices AG Borhane Cheikh Ahmed\n\nBrii...,"[176 72310353, 100\n64295, 100\n64295, 100\...","[stbochei@stud.h-da.de, freitag@eit.h-da.de, i...","[stbochei@stud.h-da.de, freitag@eit.h-da.de, i...","[Borhane Cheikh Ahmed, Damen, Herren, Dartiber..."
9,9,example.pdf,Lebenslauf Lisa Maria Zettier\n\nPersonliche D...,[],[],[],"[Lisa Maria Zettier, Lisa Maria Zettier, Lisa ..."


In [ ]:
df['text'][0]

In [11]:
def get_phone_numbers(text):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(text)
    return [re.sub(r'\D', '', num) for num in phone_numbers]

df['phn'] = df['text'].apply(get_phone_numbers)

In [12]:
df

,Unnamed: 0,filename,text,phones,emails,emails_spacy,names,phn
0,0,1034151.pdf,"Hovhannes Mkrtchyan\n\nGeyerstrasse 52, 80469,...",[176 77512727],[mkrtchyan.hov.hm@gmail.com],[mkrtchyan.hov.hm@gmail.com],[Hovhannes Mkrtchyan],[1767751272]
1,1,1037402.pdf,"Birthday: August 26th, 1993\n\nHome base: Inns...",[43 676 83555],"[juergen.hahn@gisolutions.at, markus.salmhofer...","[juergen.hahn@gisolutions.at, markus.salmhofer...","[Judith Kuneth, Judith Kuneth, Jurgen Hahn, Ma...",[6768355]
2,2,1039995.pdf,Nicholas Jedamzik\n\nn.jedamzik@jedamzik-jedam...,[58609041],[n.jedamzik@jedamzik-jedamzik.de],[n.jedamzik@jedamzik-jedamzik.de],"[Nicholas Jedamzik, Marz 2022\nBTC]",[5860904]
3,3,1042050.pdf,DARKO\n\nKESEGI\n\nMSc of Economics\n\nBIRTH D...,"[381 60 0552, 381 63 596]",[darkokesegi@gmail.com],[darkokesegi@gmail.com],[],[]
4,4,1042140.pdf,Einsatzschwerpunkte\n\nConsulting\n\nMentoring...,[2021-2022],"[info@digitalfrontiers.de, info@digitalfrontie...","[info@digitalfrontiers.de, info@digitalfrontie...","[Herrn Jerat, Herr Jerat, Herr Jerat, Im Zuge ...",[0212022]
5,5,1042449.pdf,SAMI RAUSCH\n\nManagement Consultant\n\nPERSON...,[49 170 174 58],[sami.rausch@consulting-rausch.com],[sami.rausch@consulting-rausch.com],[Carve-Out],[]
6,6,1042894.pdf,Adresse:\n\nGiesestraBe 60\n12621 Berlin\n\nMi...,"[030-50 56, 030-50 56, 030-50 56, 030-50 56, 0...","[NeumannMirko@gmx.de, NeumannMirko@gmx.de, Neu...","[NeumannMirko@gmx.de, NeumannMirko@gmx.de, Neu...","[Mirko Neumann, Mirko Neumann, Mirko Neumann, ...",[]
7,7,1043487.pdf,LEBENSLAUF Eunsu AHN\n\nPERSONLICHE DATEN\n\nA...,[157 58219873],[eunsuahn.pro@gmail.com],[eunsuahn.pro@gmail.com],[],[1575821987]
8,8,1044442.pdf,Top Itservices AG Borhane Cheikh Ahmed\n\nBrii...,"[176 72310353, 100\n64295, 100\n64295, 100\...","[stbochei@stud.h-da.de, freitag@eit.h-da.de, i...","[stbochei@stud.h-da.de, freitag@eit.h-da.de, i...","[Borhane Cheikh Ahmed, Damen, Herren, Dartiber...","[1767231035, 1006429, 1006429, 1006429, 100642..."
9,9,example.pdf,Lebenslauf Lisa Maria Zettier\n\nPersonliche D...,[],[],[],"[Lisa Maria Zettier, Lisa Maria Zettier, Lisa ...",[]
